<a href="https://colab.research.google.com/github/jyrigr/MTAT.03.238-Project/blob/master/Sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://arxiv.org/abs/1805.03545

In [0]:
import numpy as np

In [0]:
sudoku1 = np.array([[8,7,6,9,0,0,0,0,0], 
                    [0,1,0,0,0,6,0,0,0], 
                    [0,4,0,3,0,5,8,0,0], 
                    [4,0,0,0,0,0,2,1,0],
                    [0,9,0,5,0,0,0,0,0],
                    [0,5,0,0,4,0,3,0,6], 
                    [0,2,9,0,0,0,0,0,8],
                    [0,0,4,6,9,0,1,7,3],
                    [0,0,0,0,0,1,0,0,4]])

In [31]:
# TEST CELL - test every little code you want to

#sudoku1[0][3]
#a = np.arange(1,10).reshape(-1, 1)
#np.sqrt(9)
#np.transpose(a)
#np.reshape(a,(9,1))
#a

#a = np.array((1,2,3))[np.newaxis, :]
#b = np.array((1,2,3))[:, np.newaxis]
#c = np.matmul(b,a)

#a = np.arange(0,9).reshape((3,3))
#a.reshape((9,1))
#a.transpose().reshape((9,1)) 
#b = np.hstack((a.reshape((9,1)), a.transpose().reshape((9,1))))
#b

#a = np.array((0,1,2))
#b = np.ones((3,3))
#c = np.array((0,9,18)).reshape(3,1)
#a*b + np.arange(0,20,9).reshape(-1,1)
#np.repeat([a],3, axis=0).flatten()
#np.repeat([a],3)

#wb=0
#hb=0
#r=3
#a = np.array((0,1,2))
#bx = np.repeat([a],r) + wb
#by = np.repeat([a],r, axis=0).flatten() + hb
#block_peers = np.hstack((bx,by))
#block_peers

#a = np.arange(0,20)
#np.where(a[1:1+r] == 1)[0][0]
#np.zeros((1,9+1)).transpose()

#a= np.arange(1,20)
#b = np.array((4,8,9))
#a[b]

a=np.array([1,0,0,0,0,1,1,0,0,0])
#np.where(a[1:] == 1)[0].shape[0]
#np.where(a[1:] == 1)[0][0]+1
np.unique(a).shape[0]


2

In [0]:
# UPDATE: using ONE-HOT vectors of shape(1,10) for value_sets. First value is 1 if cell has fixed value
# also the corrsponding index is set to 1:
# Example: if cell has a fxed value of 5
# [1,0,0,0,0,1,0,0,0,0]
#  0 1 2 3 4 5 6 7 8 9 <- indexes: index equals value of cell. 0 indicates if fixed

def initialize(sudoku):
    h,w = sudoku.shape # using w as 9
    if h != w:
        print("Not square sudoku")
        #return
    new_sudoku = np.empty((h*w, w+1))
    #new_sudoku = np.empty((h*w, w+1+20)) # if peers are added at initialization
    for i in range(w): # ith row
        for j in range(w): # jth column
            
            if sudoku[i][j] == 0:
                value_set = np.ones((1,w+1))
                value_set[0][0] = 0
            else:
                value_set = np.zeros((1,w+1))
                value_set[0][0] = 1
                value_set[0][sudoku[i][j]] = 1
            new_sudoku[i*w+j] = value_set
            #new_sudoku[i*w+j] = np.concatanate(value_set, find_peers(i*w+j, w)) # if peers are added at initialization
    return new_sudoku

In [0]:
sudoku2 = initialize(sudoku1)
#sudoku2

In [0]:
# Every cell has its peers - cells 
def find_peers(index, range):
    h = index // range
    w = index % range
    r = np.sqrt(range) # for 3x3 matrix range=9, r=3
    wb = w // r
    hb = h // r # Block index - shows in which 3x3 block the cell is
    
    hx = np.full((range,1), h) # vertical 9x1 array
    hy = np.arange(0,range).reshape(-1, 1) # make it vertical
    row_peers = np.hstack((hx,hy)) # concatenate to vertical arrays
    #print(row_peers)
    
    wx = hy
    wy = np.full((range,1), w)
    column_peers = np.hstack((wx,wy)) # concatenate to vertical arrays
    #column_peers2 = column_peers[:,0]*range + column_peers[:,1]
    #print(column_peers2)
    
    a = np.array((0,1,2))
    bx = np.repeat([a],r, axis=0).reshape(range, 1) + hb*r
    by = np.repeat(a,r).reshape(-1, 1) + wb*r
    block_peers = np.hstack((bx,by)) # concatenate to vertical arrays
    #print(block_peers)
    
    peers = np.unique(np.vstack((row_peers, column_peers, block_peers)),axis=0)
    #print(np.where((peers[:,0] == h) * (peers[:,1] == w))[0][0]) # Finds the index of the row where the cell is located
    peers = np.delete(peers, np.where((peers[:,0] == h) * (peers[:,1] == w))[0][0], axis=0)
    peers_1D = np.int8(peers[:,0]*range + peers[:,1])
    
    return peers_1D

def constraint_propagation(sudoku):
    c, r = sudoku.shape # cell index, range of values for each sell
    r = r-1 # First column refers to fixed value. IMPORTANT: if the initialize funtion is changed so that the sudoku matrix stores peers, this value need to be redefined
    change = 1
    while change == 1: # go through the loop until there has been zero changes:
        change = 0
        for i in range(c):
            peers = find_peers(i, r) # Always same output, would be okay to store in a variable. Maybe it should be added in the sudoku initialization.
            if sudoku[i][0] == 1: # Cell value is fixed
                value = np.where(sudoku[i][1:1+r] == 1)[0][0]+1
                for p in peers:
                    if sudoku[p][0] == 0 and sudoku[p][value] == 1: # if peer is not fixed, remove cell's value from its value_set
                        sudoku[p][value] = 0
                        if np.sum(sudoku[p][1:1+r]) == 1:
                            sudoku[p][0] = 1 # if peers value_set has decreased to 1 value, declare peer fixed
                        change = 1
                    elif sudoku[p][0] == 1: # if peer is fixed
                        peer_value = np.where(sudoku[p][1:1+r] == 1)[0][0]+1
                        if sudoku[i][peer_value] == 1: # if peers value is in cells value_set
                            sudoku[i][peer_value] = 0 # remove peers value from cells value_set
                            if np.sum(sudoku[i][1:1+r]) == 1:
                                sudoku[i][0] = 1 # if cells value_set has decreased to 1 value, declare cell fixed
                            change = 1           
    return sudoku


def get_cell_value(cell):
    if cell[0] == 1: # Check if cell is fixed
        values = np.where(cell[1:] == 1) # find all indexes where value equals 1, leave out the first value that indicates if cell is fixed or not
        if values[0].shape[0] == 1: # If there are more (or less) than one indeces returned, no one value can be returned
            return values[0][0] + 1 # add one to get real value of cell
    return 0 # Means not fixed, retrun 0

def check_1D(row): #can be used for row, column or block
    r = row.shape[0] # For 3x3 sudoku r=9, -> max value
    values = np.unique([get_cell_value(cell) for cell in row]) # Use get_cell_value for each cell in row, keep unique values
    # Check if there are n (9) different values, if largest = r (9) and if smallest = 1
    if values.shape[0] == r and np.amax(values) == r and np.amin(values) == 1:
        return True # If all correct, return True
    return False # If some condition was not satisified, return False

    
def check_if_solved(sudoku):
    checksum = np.unique(np.arange(1,10))
    c, r = sudoku.shape # cell index, range of values for each sell
    r = r-1 # range of values, for 3x3 sudoku r=9
    R = np.int(np.sqrt(r)) # Rank of sudoku, for 9x9 sudoku R=3
    
    for i in range(r):
        row =    i*r + np.arange(0,r,1) # every row starts with the index that is multiple of r, contains the next r values
        column = i + np.arange(0, r*r, r) # every column starts with an index from 0 to r, the indexes for next values differ by r
        # determine which block, multiply by Rank to get the start index of that block. Inside block, first row is always [0:R-1] (0,1,2 for Rank 3), add smae value to other rows. Add r for each next row start
        block =  (i//R *R*r + i%R *R) + (np.arange(0,R,1, dtype=np.int_) * np.ones((R,R),dtype=np.int_) + np.arange(0,R*r,r, dtype=np.int_).reshape(-1,1)).flatten() # 0, 1, 2 for 9x9 sudoku * 3x3 ones
        if (check_1D(sudoku[row]) and check_1D(sudoku[column]) and check_1D(sudoku[block])) == 0:
            return False
        
        #if np.sum(sudoku[row]) != 18 or np.sum(sudoku[column]) != 18 or np.sum(sudoku[block]) != 18: # Primitive check that cheks if all values are fixed.
        #    return False
    return True
        

In [0]:
def output_initial_shape(sudoku): # Convert the sudoku to original rxr shape
    c, r = sudoku.shape # cell index, range of values for each sell
    r = r-1 # remove the fixed value indicator, for 9x9 sudoku r=9
    out_sudoku = np.empty(c, dtype=np.int_) # c should equal r*r
    for i in range(c):
        out_sudoku[i] = get_cell_value(sudoku[i])
    return out_sudoku.reshape((r,r))


In [61]:
sudoku3 = constraint_propagation(sudoku2)
#sudoku2.shape
check_if_solved(sudoku3)
sudoku4 = output_initial_shape(sudoku3)

True